This notebook implements a forecast for US gas prices for the next 13 weeks (roughly one quarter), broken down by region. I am taking this as an opportunity to kick the tires on three new packages I've been excited to try out:

- [statsforecast](https://nixtla.github.io/statsforecast/)'s AutoARIMA implementation (created by [Nixtla](https://www.nixtla.io/)). Note that gas prices are ill-suited to Prophet's model which focuses on modeling seasonality that's not really present here.
- [Lineapy](https://lineapy.org/) to save data and plots, and generate a pipeline we can schedule to run regularly. This is super useful because I'm going to do a lot of extra exploratory work here that isn't really relevant to the final automation step.
- [Quarto](https://quarto.org/) the new hotness for rendering documents from Jupyter notebooks. If you're reading this and think "hey this looks nice!" it's 100% because of Quarto.

I hope it also just demonstrates what I believe to be an effective way to wrangle data in Python, using Pandas method-chaining, altair plots, black to automatically cleanup after myself

*Interesting packages I am using* 
(the rest are hidden)


In [ ]:
#%load_ext lineapy
%load_ext nb_black
from statsforecast.models import AutoARIMA

In [ ]:
#| echo: false
import requests
import altair as alt
import numpy as np
import pandas as pd
from scipy.linalg import svd

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import acf

# 1. Download and clean up data

We are going to get data from the [EIA](https://www.eia.gov/petroleum/gasdiesel/)'s website, which has weekly gas prices by region since 1993. They (sadly) make the data available in an XLS file, but fortunately Pandas can read this natively, and we can do almost all the needed cleanup in a few lines.


In [ ]:
response = requests.get("https://www.eia.gov/petroleum/gasdiesel/xls/pswrgvwall.xls")
df = pd.read_excel(
    response.content,
    sheet_name="Data 12",
    index_col=0,
    skiprows=2,
    parse_dates=["Date"],
).rename(
    columns=lambda c: c.replace("Weekly ", "").replace(
        " All Grades All Formulations Retail Gasoline Prices  (Dollars per Gallon)", ""
    )
)

Saving this dataframe in Lineapy makes it available as an artifact I can schedule to be re-created regularly.


In [ ]:
lineapy.save(df, "weekly_gas_price_data")

# 2. Tidy and explore the data

Our first step here is to tidy the data.

- convert from wide to long format
- rename columns
- clean up missing values

After that we'll do some basic EDA.


In [ ]:
df_long = (
    df.reset_index()
    .melt(id_vars=["Date"], var_name="region", value_name="price")
    .rename(columns={"Date": "week"})
    .sort_values(["region", "week"])
    .assign(
        # if we're missing one value, just use the last value
        # (happens twice)
        price=lambda x: x["price"].combine_first(x.groupby("region")["price"].shift(1)),
        # we'll forecast log(price) and then transform
        log_price=lambda x: np.log(x["price"]),
        # percentage price changes are approximately the difference in log(price)
        price_change=lambda x: (
            x["log_price"] - x.groupby("region")["log_price"].shift(1)
        ),
    )
    .query("price == price")  # filter out NAs
)

lineapy.save(df_long, "weekly_gas_price_data_long")
df_long.head()

Count how many non-missing values we have so we can get a sense of how long the time series are.


In [ ]:
df_long.groupby("region")["price"].count().reset_index().pipe(alt.Chart).encode(
    x=alt.X("price", title="Cases"), y=alt.Y("region", sort=alt.SortField("price"))
).mark_bar()

As time goes on we get prices for more regions. They gradually introduced more fine-grained geographies over time. By 2003 or so all regions are available.


In [ ]:
df_long.groupby("week")["price"].count().reset_index().pipe(alt.Chart).encode(
    x="week", y=alt.Y("price", title="Count")
).mark_line()

This plot shows the distribution of price changes. Clearly non-normal, it looks like a Laplace distribution.


In [ ]:
(
    df_long.query("price_change == price_change")
    .sample(5000)
    .pipe(alt.Chart)
    .transform_density("price_change")
    .encode(x="value:Q", y="density:Q")
    .mark_area()
)

In [ ]:
total_regions = len(df_long["region"].unique())

complete_case_date = (
    df_long.groupby("week")["price"]
    .count()
    .reset_index()
    .query(f"price == {total_regions}")["week"]
    .min()
).strftime("%Y-%m-%d")
complete_case_date

In [ ]:
(
    df_long.groupby("region")["price_change"]
    .mean()
    .reset_index()
    .assign(annual_price_change=lambda x: x["price_change"] * 52)
    .pipe(alt.Chart)
    .encode(
        x=alt.X("region", sort=alt.SortField("annual_price_change")),
        y=alt.Y("annual_price_change", title="Annual Price Growth"),
    )
    .mark_bar()
)

## Matrix factorizations are fun


In [ ]:
wide = (
    df_long.query(f"week > '{complete_case_date}'")[["week", "region", "price_change"]]
    .set_index("week")
    .pivot(columns="region", values="price_change")
)
matrix = wide.values
print(matrix.shape)
u, d, v = svd(matrix)

In [ ]:
scree_plot = (
    pd.DataFrame({"eigenvalue": d, "index": np.arange(d.shape[0])})
    .pipe(alt.Chart)
    .encode(x="index", y="eigenvalue")
    .mark_point()
)

lineapy.save(scree_plot, "scree_plot")
scree_plot

In [ ]:
components = pd.DataFrame(
    v, columns=[f"component_{i}" for i in range(v.shape[0])], index=wide.columns
).reset_index()

components_plot = (
    components.pipe(alt.Chart)
    .encode(x="component_0", y="component_1", text="region")
    .mark_text()
    .interactive()
)

lineapy.save(components_plot, "components_plot")
components_plot

## ACF plots: are price changes mean-reverting?

Here we check if changes in gas prices have any autocorrelation. If they do, it indicates there is some momentum: if prices are increasing they are likely to increase next week and we should fill up our tank as soon as possible. If there is no autocorrelation, there's no real benefit to timing our gas purchases. If the autocorrelation is negative, we may want to wait awhile after prices increase to fill up.

**Answer**: looks like there's strong positive autocorrelation lasting about 6 weeks. 


In [ ]:
region = "U.S."
auto_correlation = (
    df_long.query(f"region == '{region}'")
    .query("price_change == price_change")["price_change"]
    .pipe(acf)
)
acf_plot = (
    pd.DataFrame({"rho": auto_correlation, "lag": np.arange(auto_correlation.shape[0])})
    .pipe(alt.Chart, title=region)
    .encode(x="lag", y="rho")
    .mark_bar()
)
lineapy.save(acf_plot, "acf_plot2")
acf_plot

# 3. Make some forecasts

Note that I'm just going to hard-code these parameters here, and then use lineapy turn them into proper parameters later.


In [ ]:
H = 13
CI = 80
region = "U.S."
cutoff_date = "2022-09-21"  # "2022-05-15"
plot_start_date = "2022-01-01"

In [ ]:
region_df = df_long.query(f"region == '{region}'")
train = region_df.query(f"week < '{cutoff_date}'")
m_aa = AutoARIMA()
m_aa.fit(train["log_price"].values)

In [ ]:
raw_forecast = m_aa.predict(h=H, level=(CI,))
raw_forecast_exp = {key: np.exp(value) for key, value in raw_forecast.items()}
forecast = pd.DataFrame(raw_forecast_exp).assign(
    week=pd.date_range(train["week"].max(), periods=H, freq="W")
    + pd.Timedelta("7 days")
)
forecast = pd.concat(
    [
        forecast,
        train.tail(1)
        .rename(columns={"price": "mean"})
        .assign(**{f"lo-{CI}": lambda x: x["mean"], f"hi-{CI}": lambda x: x["mean"]}),
    ]
)

In [ ]:
uncertainty_plot = (
    forecast.pipe(alt.Chart)
    .encode(
        x="week",
        y=alt.Y(f"lo-{CI}", title="Price"),
        y2=alt.Y2(f"hi-{CI}", title="Price"),
    )
    .mark_area(opacity=0.2)
)

history_plot = (
    region_df.query(f"week >= '{plot_start_date}'")
    .pipe(alt.Chart, title=f"{region} gas price forecast (as of {cutoff_date})")
    .encode(x=alt.X("week", title="Week"), y=alt.Y("price", title="Price"))
    .mark_line()
)

forecast_plot = forecast.pipe(alt.Chart).encode(x="week", y="mean").mark_line()

cutoff_plot = (
    train.tail(1).pipe(alt.Chart).encode(x="week").mark_rule(strokeDash=[10, 2])
)

all_plots = uncertainty_plot + history_plot + forecast_plot + cutoff_plot
lineapy.save(all_plots, "gas_price_forecast")

The forecast is kind of boring actually, the recent trend has been flat so it looks unlikely prices could climb as high as they were in June or even March.


In [ ]:
all_plots

# 4. Build a pipeline

### Parameter Refactor

Here is some magic. I can take the last plot I made and directly turn it into a function that makes the plot again -- but where I can set new options about the parameters I specified!


In [ ]:
forecast_region = lineapy.get_function(
    ["gas_price_forecast"],
    input_parameters=["region", "cutoff_date", "H"],
    reuse_pre_computed_artifacts=["weekly_gas_price_data_long"],
)

In [ ]:
result = forecast_region(region="California", cutoff_date="2022-06-07", H=13)
result["gas_price_forecast"]

In [ ]:
plots = []
for region in df_long["region"].unique():
    result = forecast_region(region=region, cutoff_date="2022-09-23")
    plots.append(result["gas_price_forecast"])

In [ ]:
chart = alt.vconcat()
for i, plot in enumerate(plots):
    if i % 5 == 0:
        row = alt.hconcat()
        chart &= row
    row |= plot
chart

In [ ]:
lineapy.save(chart, "all_forecasts_plot")

In [ ]:
lineapy.to_pipeline(["all_forecasts_plot"])